In [1]:
import os
import time
import socket
import struct
import pickle

import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models

import torch.nn as nn
import torch.nn.functional as F


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

cuda:0


In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

total_batch = len(trainloader)
print(total_batch)
print("Trainset size", trainloader.dataset.data.shape)

Files already downloaded and verified
Files already downloaded and verified
12500
Trainset size (50000, 32, 32, 3)


In [3]:
def conv_bn(inp, oup, stride):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
        nn.BatchNorm2d(oup),
        nn.ReLU(inplace=True)
    )

def conv_dw(inp, oup, stride):
    return nn.Sequential(
        nn.Conv2d(inp, inp, 3, stride, 1, groups=inp, bias=False),
        nn.BatchNorm2d(inp),
        nn.ReLU(inplace=True),

        nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
        nn.BatchNorm2d(oup),
        nn.ReLU(inplace=True),
    )

In [4]:
class ClientMobileNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(ClientMobileNet, self).__init__()
        self.layer1 = conv_bn(3, 32, 2)
        self.layer2 = conv_dw(32, 64, 1)
        self.layer3 = conv_dw(64, 128, 2)
        self.layer4 = conv_dw(128, 128, 1)
        self.layer5 = conv_dw(128, 256, 2)
        self.layer6 = conv_dw(256, 256, 1)
        self.layer7 = conv_dw(256, 512, 2)
        self.layer8 = conv_dw(512, 512, 1)
        self.layer9 = conv_dw(512, 512, 1)
        self.layer10 = conv_dw(512, 512, 1)
        self.layer11 = conv_dw(512, 512, 1)
        self.layer12 = conv_dw(512, 512, 1)
        self.layer13 = conv_dw(512, 1024, 2)
        self.layer14 = conv_dw(1024, 1024, 1)
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        #x = self.layer4(x)
        #x = self.layer5(x)
        #x = self.layer6(x)
        #x = self.layer7(x)
        #x = self.layer8(x)
        #x = self.layer9(x)
        #x = self.layer10(x)
        #x = self.layer11(x)
        #x = self.layer12(x)
        #x = self.layer13(x)
        #x = self.layer14(x)
        #x = self.avg_pool(x)
        #x = x.view(-1, 1024)
        #x = self.fc(x)
        return x

client_model = ClientMobileNet(num_classes=10)
client_model = client_model.to(device)
print(client_model)

ClientMobileNet(
  (layer1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64, bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (4): B

In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(client_model.parameters(), lr=0.001, momentum=0.9)

In [6]:
def send_msg(sock, msg):
    # prefix each message with a 4-byte length in network byte order
    msg = pickle.dumps(msg)
    msg = struct.pack('>I', len(msg)) + msg
    sock.sendall(msg)

def recv_msg(sock):
    # read message length and unpack it into an integer
    raw_msglen = recvall(sock, 4)
    if not raw_msglen:
        return None
    msglen = struct.unpack('>I', raw_msglen)[0]
    # read the message data
    msg =  recvall(sock, msglen)
    msg = pickle.loads(msg)
    return msg

def recvall(sock, n):
    # helper function to receive n bytes or return None if EOF is hit
    data = b''
    while len(data) < n:
        packet = sock.recv(n - len(data))
        if not packet:
            return None
        data += packet
    return data

In [7]:
host = 'localhost'
port = 12303
s = socket.socket()
s.connect((host, port))
epoch = recv_msg(s)   # get epoch
msg = total_batch
send_msg(s, msg)   # send total_batch of train dataset

In [8]:
from tqdm import tqdm

for e in range(epoch):
    client_weights = recv_msg(s)
    client_model.load_state_dict(client_weights, strict=False)
    client_model.eval()
    for i, data in enumerate(tqdm(trainloader, ncols=100, desc='Epoch '+str(e+1))):
        x, label = data
        x = x.to(device)
        label = label.to(device)
        
        optimizer.zero_grad()
        output = client_model(x)
        client_output = output.clone().detach().requires_grad_(True)
        msg = {
            'client_output': client_output,
            'label': label
        }
        send_msg(s, msg)
        client_grad = recv_msg(s)
        output.backward(client_grad)
        optimizer.step()
    send_msg(s, client_model.state_dict())

Epoch 1: 100%|████████████████████████████████████████████████| 12500/12500 [03:25<00:00, 60.81it/s]


In [9]:
torch.save(client_model.state_dict(), 'client_model_state_dict.pth')

# Replace 'path_to_file' with your file path
file_path = 'client_model_state_dict.pth'

# Get the size of the file in bytes
file_size = os.path.getsize(file_path)

print(f"The size of the file is {file_size / 1024} kb.")


The size of the file is 12707.603515625 kb.


In [10]:
from thop import profile
x =  torch.randn(50000, 3, 32, 32)
x = x.to(device)
macs, params = profile(client_model, inputs=(x,))
print(macs, params)

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
78028800000.0 12608.0
